In [1]:
from glob import glob

from collections import defaultdict
import json
import os

In [2]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
questions = defaultdict(set)
q = []
for f in glob('base-questions-v4*/*'):
    b = os.path.split(f)[0]
    with open(f) as fopen:
        data = json.load(fopen)
    data = '\n'.join(data)
    data = data.split('\n\n')
    if len(data) == 1:
        data = data[0].split('\n')
    else:
        datas = []
        for d in data:
            splitted = d.split('\n ')
            splitted = [s.strip() for s in splitted]
            datas.extend(splitted)
        data = datas
    data = ['.'.join(d.split('.')[1:]).strip() for d in data]
    data = [d for d in data if len(d) > 3]
        
    data = set(data)
    questions[b] |= data

In [4]:
total_questions = 0
all_questions = []
for k, v in questions.items():
    total_questions += len(v)
    all_questions.extend(v)
    
total_questions

7786

In [5]:
total_questions

7786

In [6]:
all_questions = sorted(all_questions)

In [7]:
all_questions

['Ada pantun khas tak yang menggambarkan suasana pasar malam di Malaysia penuh dengan jualan makanan sedap?',
 'Ada pantun tak yang menceritakan keindahan bunga raya di taman-taman Malaysia, sebagai lambang kebanggaan dan ketahanan negara?',
 'Adakah anda dapat mencipta pantun untuk mengajak rakan-rakan menghargai keindahan alam di Taman Negara Pahang?',
 'Adakah anda dapat menghasilkan pantun tentang pentingnya pelestarian warisan sejarah di Melaka dengan menggunakan bahasa yang puitis dan menarik?',
 'Adakah anda dapat menghasilkan pantun yang mengaitkan semangat patriotisme sempena Hari Merdeka, dengan simbolisme wau bulan yang menari-nari di langit biru?',
 'Adakah anda dapat merangkai sebuah pantun tentang pentingnya persahabatan antara pelbagai bangsa di Malaysia?',
 'Adakah pantun khas yang boleh menceritakan tentang hujan lebat di petang hari yang membuatkan kanak-kanak terpaksa bermain congkak di dalam rumah?',
 'Adakah pantun khas yang dapat menggambarkan keramaian festival T

In [ ]:
!mkdir answer-question-chatgpt4-v4
# !rm -rf answer-question-chatgpt4-v3/*

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(all_questions))):
    filename = f'answer-question-chatgpt4-v4/{i}.json'
    if os.path.exists(filename):
        continue
        
    try:
        message_text = [
            {"role":"user","content": f"{all_questions[i]}, jawab dalam bahasa melayu"},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        splitted = completion.choices[0]['message']['content'].split('\n')
    except:
        splitted = None
    with open(filename, 'w') as fopen:
        json.dump(splitted, fopen)

In [12]:
qa, texts = [], []
for i in range(len(all_questions)):
    q = all_questions[i]
    
    filename = f'answer-question-chatgpt4-v4/{i}.json'
    with open(filename) as fopen:
        answer = json.load(fopen)
    
    if len(q) > 500:
        # print(q, answer)
        continue
    
    if answer is None:
        continue
        
    answer = '\n'.join(answer)
    
    qa.append({
        'question': q.strip(),
        'answer': answer.strip()
    })
    
    texts.extend([q.strip(), answer.strip()])

In [13]:
len(texts)

15466

In [ ]:
with open('answer-question-chatgpt4-v4.texts', 'w') as fopen:
    for t in set(texts):
        fopen.write(f'{json.dumps(t)}\n')

In [ ]:
!cp answer-question-chatgpt4-v4.texts ../ctranslate2

In [14]:
mapping = {}
with open('../ctranslate2/answer-question-chatgpt4-v4.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
len(mapping)

15465

In [15]:
with open('malaysian-general-qa-v4.jsonl', 'w') as fopen:
    for q in qa:
        q['question_ms'] = mapping.get(q['question'])
        q['answer_ms'] = mapping.get(q['answer'])
        fopen.write(f'{json.dumps(q)}\n')

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='malaysian-general-qa-v4.jsonl',
    path_in_repo='malaysian-general-qa-v4.jsonl',
    repo_id='mesolitica/chatgpt4-malaysian-general-qa',
    repo_type='dataset',
)

malaysian-general-qa-v4.jsonl:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/blob/main/malaysian-general-qa-v4.jsonl'